# LSTM Classification Model

In [1]:
from river_torch.classification import RollingClassifier
from river import metrics, compose, preprocessing
from river.datasets import Bikes
import torch

In [10]:
class MyModule(torch.nn.Module):

    def __init__(self, n_features):
        super().__init__()
        self.n_features=n_features
        self.lstm = torch.nn.LSTM(input_size=n_features, hidden_size=2, num_layers=1)
        self.softmax = torch.nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        output, (hn, cn) = self.lstm(X)  # lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size)
        return self.softmax(hn)

In [11]:
dataset = Bikes()
metric = metrics.Accuracy()
optimizer_fn = torch.optim.SGD

model_pipeline = compose.Select('clouds', 'humidity', 'pressure', 'temperature', 'wind')
model_pipeline |= preprocessing.StandardScaler()
model_pipeline |= RollingClassifier(
    module=MyModule,
    loss_fn="binary_cross_entropy",
    optimizer_fn=torch.optim.SGD,
    window_size=20,
    lr=1e-2,
    append_predict=True,
)

In [12]:
for x,y in dataset:
    y_pred = model_pipeline.predict_one(x)      # make a prediction
    metric = metric.update(y, y_pred)  # update the metric
    model = model_pipeline.learn_one(x, y)    # make the model learn
print(f'Accuracy: {metric.get()}')

AttributeError: 'NoneType' object has no attribute 'out_features'